In [ ]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BenchmarkTools
import Markdown; Base.showable(::MIME"text/markdown", ::Markdown.MD) = false # to make sth like """?replace"""  works for vscode backend, remember to choose Jupyter kernel
include(pkg_folder*"src/initialize.jl")
Threads.nthreads() # check number of threads

  Activating project at `~/Realizibility_index/bnc_julia`


24

In [2]:
# #binding
c=1
N = [c 1 -1]
x_sym = [:E, :S, :C]
q_sym = [:tE, :tS]
K_sym = [:K]

model = Bnc(N = N, x_sym=x_sym, q_sym=q_sym, K_sym=K_sym)

Bnc{Int8}([1 1 -1], [1 0 1; 0 1 1], 1, 3, 2, Num[E, S, C], Num[tE, tS], Num[K], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], nothing, Vertex[], Bool[], Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), -1, [0.0, 0.0, 0.0], [0.3010299956639812, 0.3010299956639812, 0.0], false, sparse([1, 2, 1, 2], [1, 2, 3, 3], [1, 1, 1, 1], 2, 3), sparse([1, 2, 1, 2], [1, 2, 3, 3], [1, 1, 1, 1], 2, 3), [[1, 3], [2, 3]], [1, 2, 3], sparse([1, 1, 1], [1, 2, 3], [1, 1, -1], 1, 3), sparse([1, 3, 2, 3, 1, 2, 3], [1, 1, 2, 2, 3, 3, 3], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0], 3, 3), [1, 3, 5, 6], [1, 2, 1, 2], [1, 2, 3, 3], [2, 4, 7], [3, 3, 3], [1, 2, 3], [1, 3], SparseArrays.UMFPACK.UmfpackLU{Float64, Int64}(SparseArrays.UMFPACK.Symbolic{Float64, Int64}(Ptr{Nothing} @0x00000000119396f0), SparseArrays.UMFPACK.Numeric{Float64, Int64}(Ptr{Nothing} @0x00000000117aee40), 3, 3, [0, 2, 4, 7], [0, 2, 1, 2, 0, 1, 2], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0], 0, SparseArrays.UMFPA

In [3]:
get_C_C0_qK!(model, [3,3])[1]

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 4 vertices found and 4 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.


3×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 -1.0  1.0    ⋅ 
  1.0   ⋅   -1.0
   ⋅    ⋅     ⋅ 

In [5]:
get_vertices_perm_dict(model)

Dict{Vector{Int8}, Int64} with 4 entries:
  [1, 3] => 2
  [3, 3] => 4
  [1, 2] => 1
  [3, 2] => 3

In [7]:
get_vertices_neighbor_mat_qK!(model)

----------------Start calculating vertices neighbor graph, It may takes a while.----------------
Done.

-------Start calculating vertices neighbor graph with qK change dir, It may takes a while.------------
Done.



4×4 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
  ⋅   1.0  1.0   ⋅ 
 1.0   ⋅    ⋅   1.0
 1.0   ⋅    ⋅   1.0
  ⋅   1.0  1.0   ⋅ 

In [8]:
get_H!(model,1)

3×3 SparseMatrixCSC{Float64, Int64} with 5 stored entries:
 1.0   ⋅     ⋅ 
  ⋅   1.0    ⋅ 
 1.0  1.0  -1.0

In [9]:
get_P_P0!(model,2)[1]

2×3 SparseMatrixCSC{Int64, Int64} with 2 stored entries:
 1  ⋅  ⋅
 ⋅  ⋅  1

In [10]:
get_H!(model,[3,3])

3×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 -1.0   1.0   ⋅ 
  1.0  -1.0   ⋅ 
   ⋅     ⋅    ⋅ 

In [11]:
calc_volume(model,[1,2])

(0.33334764018817764, 0.0009239437999507685)

In [14]:
show_condition_qK(model,[1,2],log_space=false)

2-element Vector{Num}:
 K > tS
 K > tE

In [15]:
get_C_C0_qK!(model,1)[1]

2×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
   ⋅   -1.0  1.0
 -1.0    ⋅   1.0

In [16]:
get_C_C0_qK!(model,2)[1]

2×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 1.0  -1.0    ⋅ 
 1.0    ⋅   -1.0

In [17]:
get_C_C0_x!(model,4)[1]

2×3 SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 -1   ⋅  1
  ⋅  -1  1

In [18]:
get_C_C0_qK!(model,2)[1]

2×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 1.0  -1.0    ⋅ 
 1.0    ⋅   -1.0

Make sure directly calculate asym qK space is identical to qK mapping back x using P and calc asym x space.

In [ ]:
find_all_vertices!(model) |> length

4

In [ ]:
model = Bnc(L=L_generator(10,20))

Bnc{Int8}([1 1 … 0 0; 0 1 … 0 0; … ; 2 1 … -1 0; 0 0 … 0 -1], [1 0 … 2 0; 0 1 … 1 0; … ; 0 0 … 1 1; 0 0 … 1 0], 10, 20, 10, Num[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉, x₁₀, x₁₁, x₁₂, x₁₃, x₁₄, x₁₅, x₁₆, x₁₇, x₁₈, x₁₉, x₂₀], Num[q₁, q₂, q₃, q₄, q₅, q₆, q₇, q₈, q₉, q₁₀], Num[K₁, K₂, K₃, K₄, K₅, K₆, K₇, K₈, K₉, K₁₀], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], sparse(Int64[], Int64[], Bool[], 0, 0), sparse(Int64[], Int64[], Int8[], 0, 0), sparse(Int64[], Int64[], Int8[], 0, 0), Dict{Vector{Int8}, Any}(), 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.7781512503836436, 0.9030899869919435, 0.8450980400142568, 0.7781512503836436, 0.8450980400142568, 0.6989700043360189, 0.8450980400142568, 0.7781512503836436, 0.8450980400142568, 0.8450980400142568, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], false, sparse([1, 11, 16, 17, 19, 2, 11, 12, 13, 16  …  17, 19, 20, 10, 11, 15, 16, 17, 18, 19], [1, 1, 1, 1, 1, 2

In [ ]:
get_nonsingular_vertices_idx(model)[100]

194

In [ ]:
H = get_H!(model,194)

20×20 SparseMatrixCSC{Float64, Int64} with 70 stored entries:
⎡⠵⢄⠀⠈⠀⠁⠄⠀⠀⠁⎤
⎢⣀⢀⣑⣄⣀⡀⠀⠀⠀⡀⎥
⎢⣀⡀⠀⠠⠑⢀⡀⠀⡀⡀⎥
⎢⣀⢁⣂⣄⣢⡀⠐⢄⠀⡀⎥
⎣⣄⡑⡀⠊⡐⠁⡁⠀⠑⢀⎦

In [ ]:
Base.summarysize(model)

69051352

In [ ]:
get_C_C0_qK!(model,4)[1]

0×0 SparseMatrixCSC{Float64, Int64} with 0 stored entries

In [2]:
@variables C Et 

2-element Vector{Num}:
  C
 Et

In [8]:
mt = [1 - C/Et  0  C/Et
 0 1 0
 1 1 -1]

3×3 Matrix{Num}:
 1 + (-C) / Et  0  C / Et
             0  1       0
             1  1      -1

In [11]:
mt_inv = inv(mt) .|> simplify

3×3 Matrix{Num}:
 1.0       C / (-Et)         C / Et
   0             1.0              0
 1.0  (-C + Et) / Et  (C - Et) / Et

In [12]:
mt_inv[3,2]

(-C + Et) / Et

In [4]:
M = get_M!(model,[3,3])

Start finding all vertices, it may takes a while.
Done, with 4 vertices found and 4 real vertices.


3×3 SparseMatrixCSC{Int64, Int64} with 5 stored entries:
 ⋅  ⋅   1
 ⋅  ⋅   1
 1  1  -1

In [5]:
Cx = get_C_x!(model,[3,3])

2×3 SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 -1   ⋅  1
  ⋅  -1  1

In [10]:
CqK = Cx * pinv(Array(M))

2×3 Matrix{Float64}:
 0.25  0.25  -0.5
 0.25  0.25  -0.5

In [11]:
model = Bnc(N = N_generator(5,10))

Bnc{Int8}([0 0 … 0 0; 1 1 … 0 0; … ; 0 1 … -1 0; 0 1 … 1 -1], [1 0 … 0 0; 0 1 … 1 2; … ; 0 0 … 1 1; 0 0 … 1 1], 5, 10, 5, Num[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉, x₁₀], Num[q₁, q₂, q₃, q₄, q₅], Num[K₁, K₂, K₃, K₄, K₅], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], sparse(Int64[], Int64[], Int8[], 0, 0), sparse(Int64[], Int64[], SparseVector{Int8, Int8}[], 0, 0), sparse(Int64[], Int64[], SparseVector{Float64, Int8}[], 0, 0), Dict{Vector{Int8}, Any}(), Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), -1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.3010299956639812, 0.6989700043360189, 0.0, 0.6989700043360189, 0.6989700043360189, 0.0, 0.0, 0.0, 0.0, 0.0], false, sparse([1, 2, 3, 4, 5, 4, 5, 1, 2, 4, 5, 2, 4, 5, 2, 4, 5], [1, 2, 3, 4, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 10], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1], 5, 10), sparse([1, 2, 3, 4, 5, 4, 5, 1, 2, 4, 5, 2, 4, 5, 2, 4, 5], [1, 2, 3, 4, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9,

In [12]:
model.L

5×10 Matrix{Int64}:
 1  0  0  0  0  0  1  0  0  0
 0  1  0  0  0  0  1  0  1  2
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  1  0  1  1  1
 0  0  0  0  1  1  0  1  1  1

In [16]:
vtx = find_all_vertices!(model)
vtx2 = filter(vtx) do v
    v[2]==10 && v[4] ==10 & v[5]==10
end
get_idx(model,[1,10,3,10,10])

97

In [17]:
Cx = get_C_x!(model,97)

12×10 SparseMatrixCSC{Int64, Int64} with 24 stored entries:
 1   ⋅  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  ⋅
 ⋅  -1  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
 ⋅   ⋅  ⋅  -1   ⋅   ⋅   ⋅   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅   ⋅  -1   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
 ⋅   ⋅  ⋅   ⋅  -1   ⋅   ⋅   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅   ⋅  -1   ⋅  1
 ⋅   ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1

In [18]:
M = get_M!(model,97)

10×10 SparseMatrixCSC{Int64, Int64} with 20 stored entries:
 1  ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   1
 ⋅  ⋅  1  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   1
 ⋅  ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅   1
 ⋅  ⋅  ⋅  1  1  -1   ⋅   ⋅   ⋅   ⋅
 1  1  ⋅  ⋅  ⋅   ⋅  -1   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  1  1   ⋅   ⋅  -1   ⋅   ⋅
 ⋅  1  ⋅  ⋅  ⋅   ⋅   ⋅   1  -1   ⋅
 ⋅  1  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   1  -1

In [23]:
pinv(Array(M)) |> sparse |> x->droptol!(x,1e-10)

10×10 SparseMatrixCSC{Float64, Int64} with 68 stored entries:
  1.0         ⋅          ⋅    ⋅         …    ⋅          ⋅         ⋅ 
 -0.0769231  0.153846    ⋅   0.153846       0.230769   0.384615  0.461538
   ⋅          ⋅         1.0   ⋅              ⋅          ⋅         ⋅ 
  0.0769231  0.0128205   ⋅   0.0128205      0.269231   0.115385  0.0384615
  0.0769231  0.0128205   ⋅   0.0128205      0.269231   0.115385  0.0384615
  0.153846   0.025641    ⋅   0.025641   …   0.538462   0.230769  0.0769231
  0.923077   0.153846    ⋅   0.153846       0.230769   0.384615  0.461538
  0.153846   0.025641    ⋅   0.025641      -0.461538   0.230769  0.0769231
  0.0769231  0.179487    ⋅   0.179487      -0.230769  -0.384615  0.538462
   ⋅         0.333333    ⋅   0.333333        ⋅          ⋅         ⋅ 

In [27]:
nullspace(Array(M)') |> sparse |> x->droptol!(x,1e-10)

10×2 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
   ⋅           ⋅ 
 -0.663257   -0.47619
   ⋅           ⋅ 
 -0.0807638   0.812492
  0.744021   -0.336303
   ⋅           ⋅ 
   ⋅           ⋅ 
   ⋅           ⋅ 
   ⋅           ⋅ 
   ⋅           ⋅ 

In [31]:
# q[2], q[4],q[5] shall be the same, thus
CqK = Cx * pinv(Array(M))

12×10 Matrix{Float64}:
  0.0769231  -0.153846  -2.02904e-16  …  -0.230769  -0.384615  -0.461538
  0.0769231   0.179487   1.7722e-17      -0.230769  -0.384615  -0.461538
 -0.923077    0.179487  -3.37632e-16     -0.230769  -0.384615  -0.461538
 -0.0769231   0.153846  -1.57064e-16      0.230769   0.384615  -0.538462
 -0.0769231   0.320513  -1.36777e-16     -0.269231  -0.115385  -0.0384615
 -0.153846    0.307692  -2.59213e-16  …  -0.538462  -0.230769  -0.0769231
 -0.153846    0.307692  -1.99167e-16      0.461538  -0.230769  -0.0769231
 -0.0769231   0.153846  -1.57064e-16      0.230769   0.384615  -0.538462
 -0.0769231   0.320513  -8.12662e-17     -0.269231  -0.115385  -0.0384615
 -0.153846    0.307692  -2.59213e-16     -0.538462  -0.230769  -0.0769231
 -0.153846    0.307692  -1.99167e-16  …   0.461538  -0.230769  -0.0769231
 -0.0769231   0.153846  -1.57064e-16      0.230769   0.384615  -0.538462

In [32]:
CqK[:,2] .+= CqK[:,4]
CqK[:,2] .+= CqK[:,5]

CqK[:,4] .= 0
CqK[:,5] .= 0

for i in 1:size(CqK,1)
    if any(abs.(CqK[i,:]) .> 1e-5)
        CqK[i,:] ./= findfirst(>(1e-2), abs.(CqK[i,:]))
    end
end

In [33]:
CqK

12×10 Matrix{Float64}:
  0.0769231  -0.461538  -2.02904e-16  …  -0.230769  -0.384615  -0.461538
  0.0769231   0.538462   1.7722e-17      -0.230769  -0.384615  -0.461538
 -0.923077    0.538462  -3.37632e-16     -0.230769  -0.384615  -0.461538
 -0.0769231   0.461538  -1.57064e-16      0.230769   0.384615  -0.538462
 -0.0769231   0.961538  -1.36777e-16     -0.269231  -0.115385  -0.0384615
 -0.153846    0.923077  -2.59213e-16  …  -0.538462  -0.230769  -0.0769231
 -0.153846    0.923077  -1.99167e-16      0.461538  -0.230769  -0.0769231
 -0.0769231   0.461538  -1.57064e-16      0.230769   0.384615  -0.538462
 -0.0769231   0.961538  -8.12662e-17     -0.269231  -0.115385  -0.0384615
 -0.153846    0.923077  -2.59213e-16     -0.538462  -0.230769  -0.0769231
 -0.153846    0.923077  -1.99167e-16  …   0.461538  -0.230769  -0.0769231
 -0.0769231   0.461538  -1.57064e-16      0.230769   0.384615  -0.538462